In [1]:
#!/usr/bin/env python3
"""
Script 2: Data Preprocessing và Feature Engineering
Xử lý dữ liệu, merge các file, tạo features, xử lý missing values
"""

import pandas as pd
import numpy as np
import warnings
from pathlib import Path
import os
os.chdir('..')

warnings.filterwarnings('ignore')

print("=" * 80)
print("XỬ LÝ DỮ LIỆU VÀ FEATURE ENGINEERING")
print("=" * 80)

# ============================================================================
# 1. LOAD DỮ LIỆU
# ============================================================================

print("\n" + "=" * 80)
print("BƯỚC 1: LOAD DỮ LIỆU")
print("=" * 80)

base_path = Path("./datasets")

print("\n📂 Đang load dữ liệu...")
df_metadata = pd.read_csv(base_path / "metadata.csv")
df_electricity = pd.read_csv(base_path / "electricity_cleaned.csv", parse_dates=['timestamp'])
df_weather = pd.read_csv(base_path / "weather.csv", parse_dates=['timestamp'])

print(f"✅ Metadata: {df_metadata.shape}")
print(f"✅ Electricity: {df_electricity.shape}")
print(f"✅ Weather: {df_weather.shape}")

XỬ LÝ DỮ LIỆU VÀ FEATURE ENGINEERING

BƯỚC 1: LOAD DỮ LIỆU

📂 Đang load dữ liệu...
✅ Metadata: (1636, 32)
✅ Electricity: (17544, 1579)
✅ Weather: (331166, 10)


In [2]:
df_electricity.head(5)

,timestamp,Panther_parking_Lorriane,Panther_lodging_Cora,Panther_office_Hannah,Panther_lodging_Hattie,Panther_education_Teofila,Panther_education_Jerome,Panther_retail_Felix,Panther_parking_Asia,Panther_education_Misty,...,Cockatoo_public_Caleb,Cockatoo_education_Tyler,Cockatoo_public_Shad,Mouse_health_Buddy,Mouse_health_Modesto,Mouse_lodging_Vicente,Mouse_health_Justin,Mouse_health_Ileana,Mouse_health_Estela,Mouse_science_Micheal
0,2016-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,123.200,727.575,69.200,8.8224,370.0870,10.0,282.9965,26.0,135.0,168.2243
1,2016-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,126.475,731.200,66.275,17.6449,737.8260,30.0,574.9265,51.0,265.0,336.4486
2,2016-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,127.825,724.675,64.675,17.6449,729.9255,30.0,570.2780,50.0,272.0,336.4486
3,2016-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,130.475,737.375,65.875,17.6449,722.2620,20.0,561.1470,52.0,276.0,336.4486
4,2016-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,129.675,721.150,66.275,17.6449,719.1665,30.0,564.3695,50.0,280.0,336.4486


In [3]:
print("\n" + "=" * 80)
print("BƯỚC 2: CHUYỂN ĐỔI ELECTRICITY DATA")
print("=" * 80)

print("\n📊 Chuyển đổi từ wide format sang long format...")

df_electricity_long = pd.melt(
    df_electricity,
    id_vars=['timestamp'],
    var_name='building_id',
    value_name='electricity_consumption'
)


BƯỚC 2: CHUYỂN ĐỔI ELECTRICITY DATA

📊 Chuyển đổi từ wide format sang long format...


In [4]:
# Loại bỏ NaN
df_electricity_long = df_electricity_long.dropna(subset=['electricity_consumption'])

# Chỉ giữ lại các buildings có electricity meter
buildings_with_electricity = df_metadata[df_metadata['electricity'] == 'Yes']['building_id'].tolist()
df_electricity_long = df_electricity_long[df_electricity_long['building_id'].isin(buildings_with_electricity)]

In [5]:
print(f"✅ Long format: {df_electricity_long.shape}")
print(f"   - Số buildings: {df_electricity_long['building_id'].nunique()}")
print(f"   - Số records: {len(df_electricity_long)}")

✅ Long format: (25212579, 3)
   - Số buildings: 1572
   - Số records: 25212579


In [6]:
print("\n" + "=" * 80)
print("BƯỚC 3: MERGE VỚI METADATA")
print("=" * 80)

print("\n📊 Đang merge electricity data với metadata...")

df_merged = pd.merge(
    df_electricity_long,
    df_metadata,
    on='building_id',
    how='inner'
)

print(f"✅ Sau khi merge metadata: {df_merged.shape}")
print(f"   - Số buildings: {df_merged['building_id'].nunique()}")
print(f"   - Số timestamps: {df_merged['timestamp'].nunique()}")


BƯỚC 3: MERGE VỚI METADATA

📊 Đang merge electricity data với metadata...
✅ Sau khi merge metadata: (25212579, 34)
   - Số buildings: 1572
   - Số timestamps: 17544


In [7]:
df_merged.head(5)

,timestamp,building_id,electricity_consumption,site_id,building_id_kaggle,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,sqft,...,yearbuilt,date_opened,numberoffloors,occupants,energystarscore,eui,site_eui,source_eui,leed_level,rating
0,2016-02-01 03:00:00,Panther_parking_Lorriane,3.7207,Panther,73.0,0.0,Parking,Parking Garage,36012.7,387638.0,...,2008.0,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN
1,2016-02-06 13:00:00,Panther_parking_Lorriane,2.1004,Panther,73.0,0.0,Parking,Parking Garage,36012.7,387638.0,...,2008.0,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN
2,2016-02-14 05:00:00,Panther_parking_Lorriane,3.3006,Panther,73.0,0.0,Parking,Parking Garage,36012.7,387638.0,...,2008.0,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN
3,2016-02-20 01:00:00,Panther_parking_Lorriane,3.4807,Panther,73.0,0.0,Parking,Parking Garage,36012.7,387638.0,...,2008.0,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN
4,2016-02-29 11:00:00,Panther_parking_Lorriane,2.1604,Panther,73.0,0.0,Parking,Parking Garage,36012.7,387638.0,...,2008.0,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN


In [8]:
print("\n" + "=" * 80)
print("BƯỚC 4: MERGE VỚI WEATHER DATA")
print("=" * 80)

print("\n📊 Đang merge với weather data (theo site_id và timestamp)...")

df_final = pd.merge(
    df_merged,
    df_weather,
    on=['timestamp', 'site_id'],
    how='left'
)

print(f"✅ Sau khi merge weather: {df_final.shape}")


BƯỚC 4: MERGE VỚI WEATHER DATA

📊 Đang merge với weather data (theo site_id và timestamp)...
✅ Sau khi merge weather: (25212579, 42)


In [9]:
df_final.head(5)

,timestamp,building_id,electricity_consumption,site_id,building_id_kaggle,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,sqft,...,leed_level,rating,airTemperature,cloudCoverage,dewTemperature,precipDepth1HR,precipDepth6HR,seaLvlPressure,windDirection,windSpeed
0,2016-02-01 03:00:00,Panther_parking_Lorriane,3.7207,Panther,73.0,0.0,Parking,Parking Garage,36012.7,387638.0,...,NaN,NaN,17.2,NaN,15.0,0.0,NaN,1017.1,0.0,0.0
1,2016-02-06 13:00:00,Panther_parking_Lorriane,2.1004,Panther,73.0,0.0,Parking,Parking Garage,36012.7,387638.0,...,NaN,NaN,17.8,8.0,12.8,-1.0,-1.0,1020.5,20.0,4.1
2,2016-02-14 05:00:00,Panther_parking_Lorriane,3.3006,Panther,73.0,0.0,Parking,Parking Garage,36012.7,387638.0,...,NaN,NaN,6.7,0.0,3.3,0.0,NaN,1024.1,350.0,3.6
3,2016-02-20 01:00:00,Panther_parking_Lorriane,3.4807,Panther,73.0,0.0,Parking,Parking Garage,36012.7,387638.0,...,NaN,NaN,13.3,0.0,10.0,0.0,NaN,1025.6,10.0,1.5
4,2016-02-29 11:00:00,Panther_parking_Lorriane,2.1604,Panther,73.0,0.0,Parking,Parking Garage,36012.7,387638.0,...,NaN,NaN,21.7,NaN,8.3,0.0,NaN,1023.1,NaN,1.5


In [10]:

print("\n" + "=" * 80)
print("BƯỚC 5: FEATURE ENGINEERING - THỜI GIAN")
print("=" * 80)

print("\n📊 Đang tạo các features thời gian...")

# Sắp xếp theo building_id và timestamp
df_final = df_final.sort_values(['building_id', 'timestamp']).reset_index(drop=True)

# Features thời gian cơ bản
df_final['hour'] = df_final['timestamp'].dt.hour
df_final['day_of_week'] = df_final['timestamp'].dt.dayofweek
df_final['day_of_month'] = df_final['timestamp'].dt.day
df_final['month'] = df_final['timestamp'].dt.month
df_final['year'] = df_final['timestamp'].dt.year
df_final['is_weekend'] = (df_final['day_of_week'] >= 5).astype(int)

# Tạo season feature
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df_final['season'] = df_final['month'].apply(get_season)

# Cyclical encoding cho hour và day_of_week (để model hiểu tính tuần hoàn)
df_final['hour_sin'] = np.sin(2 * np.pi * df_final['hour'] / 24)
df_final['hour_cos'] = np.cos(2 * np.pi * df_final['hour'] / 24)
df_final['day_of_week_sin'] = np.sin(2 * np.pi * df_final['day_of_week'] / 7)
df_final['day_of_week_cos'] = np.cos(2 * np.pi * df_final['day_of_week'] / 7)
df_final['month_sin'] = np.sin(2 * np.pi * df_final['month'] / 12)
df_final['month_cos'] = np.cos(2 * np.pi * df_final['month'] / 12)

print("✅ Đã tạo features thời gian cơ bản")


BƯỚC 5: FEATURE ENGINEERING - THỜI GIAN

📊 Đang tạo các features thời gian...
✅ Đã tạo features thời gian cơ bản


In [11]:
df_final.head(5)

,timestamp,building_id,electricity_consumption,site_id,building_id_kaggle,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,sqft,...,month,year,is_weekend,season,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,month_sin,month_cos
0,2016-01-01 00:00:00,Bear_assembly_Angel,237.85,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,1,2016,0,Winter,0.000000,1.000000,-0.433884,-0.900969,0.5,0.866025
1,2016-01-01 01:00:00,Bear_assembly_Angel,221.25,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,1,2016,0,Winter,0.258819,0.965926,-0.433884,-0.900969,0.5,0.866025
2,2016-01-01 02:00:00,Bear_assembly_Angel,222.50,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,1,2016,0,Winter,0.500000,0.866025,-0.433884,-0.900969,0.5,0.866025
3,2016-01-01 03:00:00,Bear_assembly_Angel,221.40,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,1,2016,0,Winter,0.707107,0.707107,-0.433884,-0.900969,0.5,0.866025
4,2016-01-01 04:00:00,Bear_assembly_Angel,224.25,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,1,2016,0,Winter,0.866025,0.500000,-0.433884,-0.900969,0.5,0.866025


In [12]:

print("\n" + "=" * 80)
print("BƯỚC 6: FEATURE ENGINEERING - LAG FEATURES")
print("=" * 80)

print("\n📊 Đang tạo lag features...")

# Lag features (điện tiêu thụ giờ trước)
df_final['electricity_lag1'] = df_final.groupby('building_id')['electricity_consumption'].shift(1)
df_final['electricity_lag24'] = df_final.groupby('building_id')['electricity_consumption'].shift(24)  # Cùng giờ ngày hôm trước
df_final['electricity_lag168'] = df_final.groupby('building_id')['electricity_consumption'].shift(168)  # Cùng giờ tuần trước

# Rolling statistics
df_final['electricity_rolling_mean_24h'] = df_final.groupby('building_id')['electricity_consumption'].transform(
    lambda x: x.rolling(window=24, min_periods=1).mean()
)
df_final['electricity_rolling_std_24h'] = df_final.groupby('building_id')['electricity_consumption'].transform(
    lambda x: x.rolling(window=24, min_periods=1).std()
)
df_final['electricity_rolling_mean_7d'] = df_final.groupby('building_id')['electricity_consumption'].transform(
    lambda x: x.rolling(window=168, min_periods=1).mean()
)

print("✅ Đã tạo lag features và rolling statistics")


BƯỚC 6: FEATURE ENGINEERING - LAG FEATURES

📊 Đang tạo lag features...
✅ Đã tạo lag features và rolling statistics


In [13]:
df_final.head(5)

,timestamp,building_id,electricity_consumption,site_id,building_id_kaggle,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,sqft,...,day_of_week_sin,day_of_week_cos,month_sin,month_cos,electricity_lag1,electricity_lag24,electricity_lag168,electricity_rolling_mean_24h,electricity_rolling_std_24h,electricity_rolling_mean_7d
0,2016-01-01 00:00:00,Bear_assembly_Angel,237.85,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,-0.433884,-0.900969,0.5,0.866025,NaN,NaN,NaN,237.85,NaN,237.85
1,2016-01-01 01:00:00,Bear_assembly_Angel,221.25,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,-0.433884,-0.900969,0.5,0.866025,237.85,NaN,NaN,229.55,11.737973,229.55
2,2016-01-01 02:00:00,Bear_assembly_Angel,222.50,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,-0.433884,-0.900969,0.5,0.866025,221.25,NaN,NaN,227.20,9.244323,227.20
3,2016-01-01 03:00:00,Bear_assembly_Angel,221.40,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,-0.433884,-0.900969,0.5,0.866025,222.50,NaN,NaN,225.75,8.085893,225.75
4,2016-01-01 04:00:00,Bear_assembly_Angel,224.25,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,-0.433884,-0.900969,0.5,0.866025,221.40,NaN,NaN,225.45,7.034646,225.45


In [14]:
print("\n" + "=" * 80)
print("BƯỚC 7: XỬ LÝ MISSING VALUES")
print("=" * 80)

print("\n📊 Đang xử lý missing values...")

# Xác định các features cần xử lý
continuous_features = [
    'sqm', 'yearbuilt', 'numberoffloors', 'occupants',
    'airTemperature', 'cloudCoverage', 'dewTemperature', 
    'windSpeed', 'seaLvlPressure', 'precipDepth1HR'
]

categorical_features = [
    'primaryspaceusage', 'sub_primaryspaceusage', 
    'site_id', 'timezone', 'season'
]

# Fill missing values cho continuous features
for col in continuous_features:
    if col in df_final.columns:
        # Fill bằng median theo building_id nếu có thể, nếu không thì fill bằng median tổng thể
        if col in ['sqm', 'yearbuilt', 'numberoffloors', 'occupants']:
            # Features của building (không đổi theo thời gian)
            df_final[col] = df_final.groupby('building_id')[col].transform(
                lambda x: x.fillna(x.median() if not x.isna().all() else 0)
            )
            # Nếu vẫn còn NaN, fill bằng median tổng thể
            df_final[col] = df_final[col].fillna(df_final[col].median() if not df_final[col].isna().all() else 0)
        else:
            # Features thời tiết - fill bằng median theo site_id
            df_final[col] = df_final.groupby('site_id')[col].transform(
                lambda x: x.fillna(x.median() if not x.isna().all() else 0)
            )
            # Nếu vẫn còn NaN, fill bằng median tổng thể
            df_final[col] = df_final[col].fillna(df_final[col].median() if not df_final[col].isna().all() else 0)

# Fill missing values cho categorical features
for col in categorical_features:
    if col in df_final.columns:
        df_final[col] = df_final[col].fillna(
            df_final[col].mode()[0] if len(df_final[col].mode()) > 0 else 'Unknown'
        )

# Fill missing values cho lag features (bằng 0 hoặc giá trị hiện tại)
for col in ['electricity_lag1', 'electricity_lag24', 'electricity_lag168']:
    if col in df_final.columns:
        df_final[col] = df_final[col].fillna(0)

for col in ['electricity_rolling_mean_24h', 'electricity_rolling_std_24h', 'electricity_rolling_mean_7d']:
    if col in df_final.columns:
        df_final[col] = df_final[col].fillna(df_final['electricity_consumption'])

print("✅ Đã xử lý missing values")

# Kiểm tra missing values còn lại
missing_after = df_final.isnull().sum().sum()
print(f"   - Missing values còn lại: {missing_after}")


BƯỚC 7: XỬ LÝ MISSING VALUES

📊 Đang xử lý missing values...
✅ Đã xử lý missing values
   - Missing values còn lại: 404179302


In [15]:
df_final.head(5)

,timestamp,building_id,electricity_consumption,site_id,building_id_kaggle,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,sqft,...,day_of_week_sin,day_of_week_cos,month_sin,month_cos,electricity_lag1,electricity_lag24,electricity_lag168,electricity_rolling_mean_24h,electricity_rolling_std_24h,electricity_rolling_mean_7d
0,2016-01-01 00:00:00,Bear_assembly_Angel,237.85,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,-0.433884,-0.900969,0.5,0.866025,0.00,0.0,0.0,237.85,237.850000,237.85
1,2016-01-01 01:00:00,Bear_assembly_Angel,221.25,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,-0.433884,-0.900969,0.5,0.866025,237.85,0.0,0.0,229.55,11.737973,229.55
2,2016-01-01 02:00:00,Bear_assembly_Angel,222.50,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,-0.433884,-0.900969,0.5,0.866025,221.25,0.0,0.0,227.20,9.244323,227.20
3,2016-01-01 03:00:00,Bear_assembly_Angel,221.40,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,-0.433884,-0.900969,0.5,0.866025,222.50,0.0,0.0,225.75,8.085893,225.75
4,2016-01-01 04:00:00,Bear_assembly_Angel,224.25,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,-0.433884,-0.900969,0.5,0.866025,221.40,0.0,0.0,225.45,7.034646,225.45


In [16]:
print("\n" + "=" * 80)
print("BƯỚC 8: LỌC DỮ LIỆU")
print("=" * 80)

print(f"\n📊 Dữ liệu trước khi lọc: {len(df_final)} records")

# Loại bỏ các dòng có electricity_consumption <= 0 hoặc quá lớn (outliers)
# Giữ lại các giá trị hợp lý (0 < consumption < percentile 99.9)
q99_9 = df_final['electricity_consumption'].quantile(0.999)
df_final = df_final[
    (df_final['electricity_consumption'] > 0) & 
    (df_final['electricity_consumption'] < q99_9)
].copy()

print(f"✅ Dữ liệu sau khi lọc: {len(df_final)} records")
print(f"   - Đã loại bỏ: {missing_after} records có vấn đề")


BƯỚC 8: LỌC DỮ LIỆU

📊 Dữ liệu trước khi lọc: 25212579 records
✅ Dữ liệu sau khi lọc: 25187366 records
   - Đã loại bỏ: 404179302 records có vấn đề


In [17]:

print("\n" + "=" * 80)
print("BƯỚC 9: LƯU DỮ LIỆU")
print("=" * 80)

output_path = Path("./output/processed_data.parquet")
os.makedirs('output', exist_ok=True)

print(f"\n📊 Đang lưu dữ liệu đã xử lý...")
df_final.to_parquet(output_path, index=False, compression='snappy')

print(f"✅ Đã lưu vào: {output_path}")
print(f"   - Shape: {df_final.shape}")
print(f"   - Columns: {len(df_final.columns)}")

# Lưu thông tin về features
features_info = {
    'continuous_features': [f for f in continuous_features if f in df_final.columns],
    'categorical_features': [f for f in categorical_features if f in df_final.columns],
    'time_features': ['hour', 'day_of_week', 'month', 'year', 'is_weekend', 'season',
                      'hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos', 
                      'month_sin', 'month_cos'],
    'lag_features': ['electricity_lag1', 'electricity_lag24', 'electricity_lag168',
                     'electricity_rolling_mean_24h', 'electricity_rolling_std_24h', 
                     'electricity_rolling_mean_7d'],
    'target': 'electricity_consumption'
}

import json
with open('output/features_info.json', 'w') as f:
    json.dump(features_info, f, indent=2)

print(f"✅ Đã lưu thông tin features vào: output/features_info.json")

# Tóm tắt
print("\n" + "=" * 80)
print("TÓM TẮT")
print("=" * 80)
print(f"✅ Đã xử lý xong dữ liệu!")
print(f"   - Tổng số records: {len(df_final)}")
print(f"   - Số buildings: {df_final['building_id'].nunique()}")
print(f"   - Số features: {len(df_final.columns)}")
print(f"   - Khoảng thời gian: {df_final['timestamp'].min()} đến {df_final['timestamp'].max()}")
print(f"\n📁 Dữ liệu đã được lưu vào: {output_path}")




BƯỚC 9: LƯU DỮ LIỆU

📊 Đang lưu dữ liệu đã xử lý...
✅ Đã lưu vào: output/processed_data.parquet
   - Shape: (25187366, 61)
   - Columns: 61
✅ Đã lưu thông tin features vào: output/features_info.json

TÓM TẮT
✅ Đã xử lý xong dữ liệu!
   - Tổng số records: 25187366
   - Số buildings: 1572
   - Số features: 61
   - Khoảng thời gian: 2016-01-01 00:00:00 đến 2017-12-31 23:00:00

📁 Dữ liệu đã được lưu vào: output/processed_data.parquet
